In [25]:
#Instaling packages
#!pip install category_encoders
#!pip install pandas
#!pip install scikit-learn
#!unzip arquive.zip

In [26]:
#Importing the required libraries
import pandas as pd
import numpy as np
import pickle #Pickling” is the process whereby a Python object hierarchy is converted into a byte stream
import warnings

warnings.simplefilter("ignore")

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_percentage_error,
    mean_absolute_error)

from category_encoders import TargetEncoder
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV


In [27]:
sklearn.__version__  #to use in requeriments.txt

'1.2.1'

In [28]:
#Defining a function to load the training and test data into pandas DataFrames
def load_data(train_path:str, test_path:str) -> (pd.DataFrame, pd.DataFrame):
    """
    Loads the train and test data into pandas DataFrames
    """

    train = pd.read_csv(train_path)
    
    #adding new features to the model
    train = train.join(pd.get_dummies(train.sector)).drop(['sector'],axis=1) #placing the "type" category in the training set
    train = train.join(pd.get_dummies(train.type)).drop(['type'],axis=1)
    train['bathroom_ratio'] = train['n_bathroom']/train['n_rooms']
    train['household_rooms']=train['n_rooms']/(train['casa']+train['departamento'])
    train = train.dropna()
    #feature engineering

    test = pd.read_csv(test_path)
    
    #adding new features to the model
    test=test.join(pd.get_dummies(test.sector)).drop(['sector'],axis=1)
    test = test.join(pd.get_dummies(test.type)).drop(['type'],axis=1)
    test['bathroom_ratio'] = test['n_bathroom']/test['n_rooms']
    test['household_rooms']=test['n_rooms']/(test['casa']+test['departamento'])
    test = test.dropna()
  
    
    return train, test
#Load the training and test data
train, test = load_data('train.csv', 'test.csv')


In [29]:
#Specifing the columns to use for training, the categorical columns, and the target variable:
train_cols = [
    col for col in train.columns if col not in ['id', 'target']
    ]

#|categorical_cols = ["type", "sector"]
#categorical_cols = ["type"]
target           = "price"

In [30]:
#Defining the categorical transformer using TargetEncoder
categorical_transformer = TargetEncoder()

#Create a ColumnTransformer to apply the categorical transformer to the categorical columns
#preprocessor = ColumnTransformer(
##    transformers=[
#        ('categorical',
#          categorical_transformer,
#          categorical_cols)
#    ])

#Defining the steps of the pipeline, including the preprocessor and the model (GradientBoostingRegressor)
steps = [
 #   ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(**{
        "learning_rate":0.03, #alpha
        "n_estimators":2500,   #m
        "max_depth":6,
        "loss":"absolute_error"
    }))
]
#Creating the pipeline using the defined steps
pipeline = Pipeline(steps)


In [31]:
#Fitting the pipeline on the training data:
pipeline.fit(train[train_cols], train[target])

Pipeline(steps=[('model',
                 GradientBoostingRegressor(learning_rate=0.03,
                                           loss='absolute_error', max_depth=6,
                                           n_estimators=2500))])

In [32]:
#Predicting property valuations for the test data using the trained model:
test_predictions = pipeline.predict(test[train_cols])
test_target = test[target].values

In [33]:
type(test_predictions), type(test_target)

(numpy.ndarray, numpy.ndarray)

In [34]:
#Model Evaluation
#defining a function to print evaluation metrics:
def print_metrics(predictions, target):
    print("RMSE: ", np.sqrt(mean_squared_error(predictions, target)))
    print("MAPE: ", mean_absolute_percentage_error(predictions, target))
    print("MAE : ", mean_absolute_error(predictions, target))

In [35]:
print_metrics(test_predictions, test_target)

RMSE:  372.90731094951246
MAPE:  0.0007596856171597557
MAE :  15.175179419249107


Exporting the model

In [36]:
with open('trained_pipeline-0.1.0.pkl','wb') as b:
    pickle.dump(pipeline, b)

In [37]:
!zip -r ./trained_pipeline_model-0.1.0.pkl.zip ./trained_pipeline-0.1.0.pkl

'zip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [38]:
pipeline.predict(train.loc[[0]])

array([11899.99960352])

In [51]:
print(type(train[0:1]))
pipeline.predict(train[0:1])

<class 'pandas.core.frame.DataFrame'>


array([11899.99960352])

In [64]:
# net_usable_area  net_area  n_rooms  n_bathroom  latitude  longitude  price  \
#la reina  las condes  lo barnechea  nunoa  providencia  vitacura   casa  \
#departamento  bathroom_ratio  household_rooms  
apicsv = pd.read_csv("api.csv",sep=",")
print(apicsv)
pipeline.predict(apicsv.loc[[0]])

   net_usable_area  net_area  n_rooms  n_bathroom  latitude  longitude  price  \
0            140.0     170.0      4.0         4.0 -33.40123  -70.58056  11900   

   la reina  las condes  lo barnechea  nunoa  providencia  vitacura  casa  \
0         0           0             0      0            0         1     0   

   departamento  bathroom_ratio  household_rooms  
0             1               1                4  


array([11899.99960352])

In [65]:
with open(f"trained_pipeline-0.1.0.pkl","rb") as f:
    model = pickle.load(f) #load the model
model.predict(apicsv.loc[[0]])

array([11899.99960352])

TypeError: float() argument must be a string or a real number, not 'dict'